# 一.安装PaddleSeg

PaddleSeg是基于飞桨PaddlePaddle的端到端图像分割套件，内置40+模型算法及140+预训练模型，支持配置化驱动和API调用开发方式，打通数据标注、模型开发、训练、压缩、部署的全流程，提供语义分割、交互式分割、Matting、全景分割四大分割能力，助力算法在医疗、工业、遥感、娱乐等场景落地应用。

![](https://ai-studio-static-online.cdn.bcebos.com/05073260b0c3489394026c375b88017e896024aafefb4c6bafb479e8d9236608)



In [1]:
# 切换到工作目录
%cd /home/aistudio/work

# 解压下载好的PaddleSeg
!unzip PaddleSeg.zip

# 安装PaddleSeg依赖
%cd PaddleSeg
!pip install -r requirements.txt


  inflating: PaddleSeg/paddleseg/models/sinet.py  
   creating: PaddleSeg/paddleseg/models/layers/
  inflating: PaddleSeg/paddleseg/models/layers/attention.py  
  inflating: PaddleSeg/paddleseg/models/layers/pyramid_pool.py  
   creating: PaddleSeg/paddleseg/models/layers/__pycache__/
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/nonlocal2d.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/wrap_functions.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/tensor_fusion_helper.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/activation.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/tensor_fusion.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/layer_libs.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/pyramid_pool.cpython-37.pyc  
  inflating: PaddleSeg/paddleseg/models/layers/__pycache__/attention.cpython-37.pyc  


In [2]:
# 删除多余文件
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Image
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Image_Masks2
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/Layer_Masks
!rm -rf /home/aistudio/work/PaddleSeg/data/mine/test

# 二.数据集处理

PaddleSeg使用单通道的标注图片(灰度标注图)，每一种像素值代表一种类别，像素标注类别需要从0开始递增，例如0，1，2，3表示有4种类别。

建议标注图像使用PNG无损压缩格式的图片，支持的标注类别最多为256类。


通常标注图片是单通道的灰度图，显示是全黑效果，无法直接观察标注是否正确。 在原来的灰度标注图中注入调色板，就可以得到伪彩色的标注图。 灰度标注图和伪彩色标注图的对比如下。

![](https://ai-studio-static-online.cdn.bcebos.com/8057bf7f10864a1d9b8068d9b7a923dafd8006328579471d9e97ae8c9082e657)



In [3]:
# 解压数据集
%cd /home/aistudio/data/data170514

!unzip -o GOALS2022-Train.zip
!unzip -o GOALS2022-Validation.zip


# 移动到PaddleSeg/data/mine 目录下 - 训练集
! mv Train/Image /home/aistudio/work/PaddleSeg/data/mine/
! mv Train/Layer_Masks /home/aistudio/work/PaddleSeg/data/mine/

# 移动到PaddleSeg/data/mine 目录下 - 测试集
! mv GOALS2022-Validation/Image /home/aistudio/work/PaddleSeg/data/mine/test


/home/aistudio/data/data170514
Archive:  GOALS2022-Train.zip
   creating: Train/
   creating: Train/Layer_Masks/
  inflating: Train/.DS_Store         
  inflating: __MACOSX/Train/._.DS_Store  
   creating: Train/Image/
   creating: Train/Layer_show/
  inflating: Train/Train_GC_GT.xlsx  
  inflating: __MACOSX/Train/._Train_GC_GT.xlsx  
  inflating: Train/License-GOALS-0606.pdf  
  inflating: __MACOSX/Train/._License-GOALS-0606.pdf  
  inflating: Train/Layer_Masks/0065.png  
  inflating: Train/Layer_Masks/0071.png  
  inflating: Train/Layer_Masks/0059.png  
  inflating: Train/Layer_Masks/0058.png  
  inflating: Train/Layer_Masks/0070.png  
  inflating: Train/Layer_Masks/0064.png  
  inflating: Train/Layer_Masks/0099.png  
  inflating: Train/Layer_Masks/0072.png  
  inflating: Train/Layer_Masks/0066.png  
  inflating: Train/Layer_Masks/0067.png  
  inflating: Train/Layer_Masks/0073.png  
  inflating: Train/Layer_Masks/0098.png  
  inflating: Train/Layer_Masks/0088.png  
  inflating: Train

生成train.txt和valid.txt 和 灰度标注图

In [4]:
import cv2
class Pre(object):
    def __init__(self):
        self.dataPath = '/home/aistudio/work/PaddleSeg/data/mine/'
    def run(self):
        # 生成train.txt和valid.txt
        trainFile = open(self.dataPath + 'train.txt', 'w')
        vaildFile = open(self.dataPath + 'valid.txt', 'w')
        for i in range(1,101):
            name = str(i).zfill(4)
            if i <= 90:
                trainFile.write('Image/'+name+'.png Image_Label/'+name+'.png')
                trainFile.write('\n')
            else:
                vaildFile.write('Image/'+name+'.png Image_Label/'+name+'.png')
                vaildFile.write('\n')
            
            # 将标签图片转为标准标准图片
            image = cv2.imread(self.dataPath+'Layer_Masks/'+name+'.png')

            # 像素值为0的是RNFL(类别 0)
            # 像素值为80的是GCIPL(类别 1)
            # 像素值为160的是脉络膜(类别 2)
            # 像素值为255的是其他（类别3）。
            image[image == 80] = 1
            image[image == 160] = 2
            image[image == 255] = 3
            image = image[:,:,1]
            cv2.imwrite(self.dataPath+'Image_Label/'+name+'.png', image)

pre = Pre()
pre.run()

In [5]:
# 灰度标注图转换为伪彩色标注图 （非必须）
%cd /home/aistudio/work/PaddleSeg

!python tools/gray2pseudo_color.py  data/mine/Image_Label data/mine/Image_Masks2

/home/aistudio/work/PaddleSeg
Creating colorful label directory: data/mine/Image_Masks2
New label path: data/mine/Image_Masks2/0024.png
New label path: data/mine/Image_Masks2/0058.png
New label path: data/mine/Image_Masks2/0074.png
New label path: data/mine/Image_Masks2/0002.png
New label path: data/mine/Image_Masks2/0029.png
New label path: data/mine/Image_Masks2/0065.png
New label path: data/mine/Image_Masks2/0040.png
New label path: data/mine/Image_Masks2/0051.png
New label path: data/mine/Image_Masks2/0041.png
New label path: data/mine/Image_Masks2/0048.png
New label path: data/mine/Image_Masks2/0045.png
New label path: data/mine/Image_Masks2/0099.png
New label path: data/mine/Image_Masks2/0075.png
New label path: data/mine/Image_Masks2/0063.png
New label path: data/mine/Image_Masks2/0022.png
New label path: data/mine/Image_Masks2/0042.png
New label path: data/mine/Image_Masks2/0015.png
New label path: data/mine/Image_Masks2/0071.png
New label path: data/mine/Image_Masks2/0039.png


# 三.训练

本项目选用PP-LiteSeg模型训练学习
![](https://ai-studio-static-online.cdn.bcebos.com/91607bc98acb42b88a50c659b63cdb6b4ffa6dda82ec45548fa65949eba4f6b5)

配置文件在configs/qick_start/mine.yml中。其他模型的配置方案可以参考 configs目录下其他文件

```
batch_size: 10  #设定batch_size的值即为迭代一次送入网络的图片数量，一般显卡显存越大，batch_size的值可以越大。如果使用多卡训练，总得batch size等于该batch size乘以卡数。
iters: 2000    #模型训练迭代的轮数

train_dataset:  #训练数据设置
  type: Dataset #指定加载数据集的类
  dataset_root: data/mine #数据集路径
  train_path: data/mine/train.txt  #数据集中用于训练的标识文件
  num_classes: 4  #指定类别个数（背景也算为一类）
  mode: train #表示用于训练
  transforms: #模型训练的数据预处理方式。
    - type: ResizeStepScaling #将原始图像和标注图像随机缩放为0.5~2.0倍
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop #从原始图像和标注图像中随机裁剪512x512大小
      crop_size: [512, 512]
    - type: RandomHorizontalFlip  #对原始图像和标注图像随机进行水平反转
    - type: RandomDistort #对原始图像进行亮度、对比度、饱和度随机变动，标注图像不变
      brightness_range: 0.5
      contrast_range: 0.5
      saturation_range: 0.5
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

val_dataset:  #验证数据设置
  type: Dataset #指定加载数据集的类
  dataset_root: data/mine #数据集路径
  val_path: data/mine/valid.txt  #数据集中用于验证的标识文件
  num_classes: 2  #指定类别个数（背景也算为一类）
  mode: val #表示用于验证
  transforms: #模型验证的数据预处理的方式
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

optimizer: #设定优化器的类型
  type: sgd #采用SGD（Stochastic Gradient Descent）随机梯度下降方法为优化器
  momentum: 0.9 #设置SGD的动量
  weight_decay: 4.0e-5 #权值衰减，使用的目的是防止过拟合

lr_scheduler: # 学习率的相关设置
  type: PolynomialDecay # 一种学习率类型。共支持12种策略
  learning_rate: 0.01 # 初始学习率
  power: 0.9
  end_lr: 0

loss: #设定损失函数的类型
  types:
    - type: CrossEntropyLoss  #CE损失
  coef: [1, 1, 1] # PP-LiteSeg有一个主loss和两个辅助loss，coef表示权重，所以 total_loss = coef_1 * loss_1 + .... + coef_n * loss_n

model:  #模型说明
  type: PPLiteSeg  #设定模型类别
  backbone:  # 设定模型的backbone，包括名字和预训练权重
    type: STDC2
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet2.tar.gz
```

In [6]:
%cd /home/aistudio/work/PaddleSeg/

!export CUDA_VISIBLE_DEVICES=0 # 设置1张可用的卡

!python train.py \
       --config configs/quick_start/mine.yml \
       --save_interval 500 \
       --do_eval \
       --use_vdl \
       --save_dir output

/home/aistudio/work/PaddleSeg
/home/aistudio/work/PaddleSeg/paddleseg/models/losses/rmi_loss.py:78: DeprecationWarning: invalid escape sequence \i
  """
2022-11-17 20:04:35 [INFO]	
------------Environment Information-------------
platform: Linux-4.15.0-140-generic-x86_64-with-debian-stretch-sid
Python: 3.7.4 (default, Aug 13 2019, 20:35:49) [GCC 7.3.0]
Paddle compiled with cuda: True
NVCC: Build cuda_11.2.r11.2/compiler.29618528_0
cudnn: 8.2
GPUs used: 1
CUDA_VISIBLE_DEVICES: None
GPU: ['GPU 0: Tesla V100-SXM2-32GB']
GCC: gcc (Ubuntu 7.5.0-3ubuntu1~16.04) 7.5.0
PaddleSeg: 2.6.0
PaddlePaddle: 2.3.2
OpenCV: 4.6.0
------------------------------------------------
2022-11-17 20:04:35 [INFO]	
---------------Config Information---------------
batch_size: 10
iters: 2000
loss:
  coef:
  - 1
  - 1
  - 1
  types:
  - ignore_index: 255
    type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  backbone:
    pretrained: https://bj.bcebos.

# 四.评估

In [7]:
!python val.py \
       --config configs/quick_start/mine.yml \
       --model_path output/best_model/model.pdparams

2022-11-17 20:28:18 [INFO]	
---------------Config Information---------------
batch_size: 10
iters: 2000
loss:
  coef:
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet2.tar.gz
    type: STDC2
  type: PPLiteSeg
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/mine
  mode: train
  num_classes: 4
  train_path: data/mine/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 512
    - 512
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_range: 0.5
    contrast_range: 0.5
    saturation_range: 0.5
    type: RandomDistort
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: data/mine
  mode: val
  num_classes: 4
  transforms:
  - type: Norm

# 五.预测生成图片

In [9]:
!python predict.py \
       --config configs/quick_start/mine.yml \
       --model_path output/best_model/model.pdparams \
       --image_path data/mine/test/ \
       --save_dir output/result \
       --custom_color 0 0 0 80 80 80 160 160 160 255 255 255

2022-11-17 20:35:42 [INFO]	
---------------Config Information---------------
batch_size: 10
iters: 2000
loss:
  coef:
  - 1
  - 1
  - 1
  types:
  - type: CrossEntropyLoss
lr_scheduler:
  end_lr: 0
  learning_rate: 0.01
  power: 0.9
  type: PolynomialDecay
model:
  backbone:
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet2.tar.gz
    type: STDC2
  type: PPLiteSeg
optimizer:
  momentum: 0.9
  type: sgd
  weight_decay: 4.0e-05
train_dataset:
  dataset_root: data/mine
  mode: train
  num_classes: 4
  train_path: data/mine/train.txt
  transforms:
  - max_scale_factor: 2.0
    min_scale_factor: 0.5
    scale_step_size: 0.25
    type: ResizeStepScaling
  - crop_size:
    - 512
    - 512
    type: RandomPaddingCrop
  - type: RandomHorizontalFlip
  - brightness_range: 0.5
    contrast_range: 0.5
    saturation_range: 0.5
    type: RandomDistort
  - type: Normalize
  type: Dataset
val_dataset:
  dataset_root: data/mine
  mode: val
  num_classes: 4
  transforms:
  - type: Norm

# 参考项目
https://aistudio.baidu.com/aistudio/projectdetail/4609530?channelType=0&channel=0